# Geometry evolution models in OGGM

Until now, we have only looked at the glacier in a static way. In today's session, we will begin exploring how the glacier geometry dynamically evolves over time. Finally, we will revisit the calibration procedure, this time also considering the effects of a dynamically evolving glacier geometry.

## OGGM setup

With the tasks we completed in the last session, we have finished everything needed from the level 2 preprocessed directories, and we can now start this session from level 3.

In [ ]:
from oggm import cfg, utils, workflow, tasks
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import xarray as xr
import pandas as pd
import numpy as np

In [ ]:
cfg.initialize()

# define working directory
path = 'evolution_working_dir'
utils.mkdir(path, reset=False)  # if you set reset=True, everything will be deleted and you can start from a fresh state
cfg.PATHS['working_dir'] = path

In [ ]:
# select the glacier of your choice, you can use the Glims viewer from the first session
rgi_ids = ['RGI60-11.01450']  # Aletsch

# we load the outline data from the oggm cluster
prepro_base_url_L3 = 'https://cluster.klima.uni-bremen.de/~oggm/gdirs/oggm_v1.6/L3-L5_files/2023.3/elev_bands/W5E5/'
gdirs = workflow.init_glacier_directories(rgi_ids,
                                          from_prepro_level=3,  # here we select level 3
                                          prepro_base_url=prepro_base_url_L3,
                                          prepro_border=80,  # could be 10, 80, 160 or 240
                                         )
gdir = gdirs[0]  # for convenience, we define a single variable for our glacier

Let's see what the diagnostics tell us:

In [ ]:
gdir.get_diagnostics()

<div class="alert alert-warning">
    <b>Task</b>: Do you remember the meaning of all variables?
</div>

Your answer here:

## A first dynamic run

For some theoretical background for this session, please first read the ["Ice flow" and "Bed shapes" sections of the OGGM documentation](https://docs.oggm.org/en/stable/ice-dynamics.html).

In [ ]:
cfg.PARAMS['store_model_geometry'] = True

In [ ]:
workflow.execute_entity_task(tasks.run_from_climate_data, gdirs,
                             output_filesuffix='_first_run');

Okay, that was easy. Now let's have a look at our run:

In [ ]:
# Open the geometry of the glacier after the run
from oggm.core import flowline
fmod_first_run = flowline.FileModel(gdir.get_filepath('model_geometry', filesuffix='_first_run'))

# prepare the data for easy plotting
fl = fmod_first_run.fls[-1]  # Main flowline
df_coords = pd.DataFrame(index=fl.dis_on_line*gdir.grid.dx)
df_coords.index.name = 'Distance along flowline'
df_coords['bed_elevation'] = fl.bed_h

# read out the surface height for each year
df_surf_h = pd.DataFrame(index=df_coords.index, columns=fmod_first_run.years, dtype=np.float64)
for year in fmod_first_run.years:
    fmod_first_run.run_until(year)
    fl = fmod_first_run.fls[-1]
    df_surf_h[year] = fl.surface_h

In [ ]:
years_to_plot = [2004, 2012, 2020]
f, ax = plt.subplots()

df_surf_h[years_to_plot].plot(ax=ax);
df_coords['bed_elevation'].plot(ax=ax, color='k');
ax.set_title('Glacier elevation at three points in time')
ax.legend(title='Year');

<div class="alert alert-warning">
    <b>Task</b>: What do you observe from in this plot? Why is the model run starting from 2004?
</div>

Your answer here:

Now have a look at the total volume and area and compare it to our calibration data:

In [ ]:
# load the model run output
ds_first_run = utils.compile_run_output(gdirs, input_filesuffix='_first_run')

In [ ]:
# loading the volume estimate
estimated_volume = pd.read_hdf(utils.get_demo_file('rgi62_itmix_df.h5')).loc[gdir.rgi_id]['vol_itmix_m3']  # m³

# getting the outline area
observed_area = gdir.rgi_area_m2  # m²

In [ ]:
fig, axs = plt.subplots(2, 1, figsize=(8, 6), gridspec_kw={'hspace': 0.4})

# plot the volume evolution and the estimated volume
ax_vol = axs[0]
ds_first_run.volume.plot(ax=ax_vol, label='first model run')
ax_vol.plot(gdir.rgi_date + 1, estimated_volume, 'ro', label='estimated volume')
ax_vol.legend(); ax_vol.grid('on');

# plot the area evolution and the observed outline area
ax_area = axs[1]
ds_first_run.area.plot(ax=ax_area, label='first model run')
ax_area.plot(gdir.rgi_date + 1, observed_area, 'ro', label='observed outline area')
ax_area.legend(); ax_area.grid('on');


<div class="alert alert-warning">
    <b>Task</b>: The area is perfectly matched, however the volume not. Why? (Hint: it has to do with the bed inversion workflow.)
</div>

Your answer here:

Finally, during the calibration of the mass balance, we used the geodetic mass balance from [Hugonnet et al. (2021)](https://www.nature.com/articles/s41586-021-03436-z). So let’s also compare our first dynamic run with this dataset.

One issue we face is that the observation covers the period 2000 to 2020, while our model run only starts in 2004. However, for now, we will ignore this mismatch and calculate the average annual mass change anyway.

In [ ]:
# load the geodetic mb data
ref_period = cfg.PARAMS['geodetic_mb_period']
df_ref_dmdtda = utils.get_geodetic_mb_dataframe().loc[gdir.rgi_id]  # get the data from Hugonnet et al., 2021
df_ref_dmdtda = df_ref_dmdtda.loc[df_ref_dmdtda['period'] == ref_period]  # only select the desired period
dmdtda_reference = df_ref_dmdtda['dmdtda'].values[0] * 1000  # get the reference dmdtda and convert into kg m-2 yr-1
dmdtda_reference_error = df_ref_dmdtda['err_dmdtda'].values[0] * 1000  # corresponding uncertainty

# calculate the model counterpart
first_run_dmdtda = ((ds_first_run.volume.loc[2020].values[0] -
                     ds_first_run.volume.loc[2004].values[0]) /
                    gdir.rgi_area_m2 /
                    (2020 - 2004) *
                    cfg.PARAMS['ice_density'])

# and print the two values
print(f'Reference dmdtda 2000 to 2020 (Hugonnet 2021): {dmdtda_reference:.2f} +/- {dmdtda_reference_error:6.2f} kg m-2 yr-1')
print(f'First run dmdtda 2004 to 2020:                 {first_run_dmdtda:.2f}            kg m-2 yr-1')

But wait, during the calibration of the mass balance model, we adjusted it to perfectly match the geodetic observation.
Let’s double-check that result:

In [ ]:
from oggm.core import massbalance

# we initialize our mass balance model and calculate the mean specific mass balance over the year 2000 and 2020
mb_mod = massbalance.MonthlyTIModel(gdir)
fls = gdir.read_pickle('inversion_flowlines')
years = np.arange(2000, 2020)
mb_model_dmdtda = mb_mod.get_specific_mb(fls=fls, year=years).mean()

print(f'Mass balance dmdtda 2000 to 2020: {mb_model_dmdtda:.2f} kg m-2 yr-1')

<div class="alert alert-warning">
    <b>Task</b>: The calibrated mass balance model perfectly matches the geodetic observation, but the dynamic run does not. Why do you think that is?
</div>

Your answer here:

## Recalibrate the mass balance model including dynamics

The reason for the mismatch is that the glacier surface geometry changes over time, while we assumed a constant geometry during the initial calibration of the mass balance model.

To account for this, OGGM includes a [dynamic calibration workflow](https://docs.oggm.org/en/stable/dynamic-spinup.html). This approach starts at an earlier year in the past (by default 1979) and searches for a melt factor that allows the mass balance model to match the observed geodetic mass balance, while also considering the dynamic evolution of the glacier's surface geometry.

In [ ]:
tasks.run_dynamic_melt_f_calibration(gdir,
                                     ys=1979,  # When to start the spinup
                                     ye=2020,  # When the simulation should stop
                                     output_filesuffix='_dynamic_melt_f',  # Where to write the output
                                     err_dmdtda_scaling_factor=0.2,
                                    );

In [ ]:
ds_spinup = utils.compile_run_output(gdirs, input_filesuffix='_dynamic_melt_f')

In [ ]:
# calculate the model counterpart
spinup_dmdtda = ((ds_spinup.volume.loc[2020].values[0] -
                  ds_spinup.volume.loc[2000].values[0]) /
                 gdir.rgi_area_m2 /
                 (2020 - 2000) *
                 cfg.PARAMS['ice_density'])

# and print the two values
print(f'Reference dmdtda 2000 to 2020 (Hugonnet 2021): {dmdtda_reference:.2f} +/- {dmdtda_reference_error:6.2f} kg m-2 yr-1')
print(f'First run dmdtda 2004 to 2020:                 {first_run_dmdtda:.2f}            kg m-2 yr-1')
print(f'Spinup run dmdtda 2000 to 2020:                {spinup_dmdtda:.2f}            kg m-2 yr-1')

Much better! Now, let’s also compare the volume and area evolution of the two runs to see how the glacier behaves under static vs. dynamic calibration:

In [ ]:
fig, axs = plt.subplots(2, 1, figsize=(8, 6), gridspec_kw={'hspace': 0.4})

# plot the volume evolution and the estimated volume
ax_vol = axs[0]
ds_first_run.volume.plot(ax=ax_vol, label='first model run')
ds_spinup.volume.plot(ax=ax_vol, label='spinup run')
ax_vol.plot(gdir.rgi_date + 1, estimated_volume, 'ro', label='estimated volume')
ax_vol.legend(); ax_vol.grid('on');

# plot the area evolution and the observed outline area
ax_area = axs[1]
ds_first_run.area.plot(ax=ax_area, label='first model run')
ds_spinup.area.plot(ax=ax_area, label='spinup run')
ax_area.plot(gdir.rgi_date + 1, observed_area, 'ro', label='observed outline area')
ax_area.legend(); ax_area.grid('on');


As you can see, the area is not matched as precisely as before, and the volume is also slightly less accurate.
However, the geodetic mass balance is now matched much more closely, which was the main goal of the dynamic calibration.

With this, we haveve reached the final level 4 of the preprocessed directories. The dynamic spinup runs we just performed are used by OGGM by default to initialize projections into the future, something we will explore in the next session.

But now that we have a dynamic glacier model, let’s take some time to explore how changing certain parameters affects the glacier’s dynamic evolution.

## Sandbox

Let's have some fun and explore how changing the individual parameters we previously calibrated affects the glacier’s evolution.

To do this, we will run the model with different parameter settings and compare the results:

In [ ]:
# First we make a run with the default settings and define this as our default
tasks.run_from_climate_data(gdir,
                            output_filesuffix='_default');

In [ ]:
# next conduct a run with doubled Glen A parameter
tasks.run_from_climate_data(gdir,
                            output_filesuffix='_glen_a_doubled',
                            glen_a_fac=2
                           );

In [ ]:
# and finally we decrease the melt factor
from functools import partial
params = gdir.read_json('mb_calib')

# here we adapt the melt factor
params['melt_f'] *= 0.95 

# you can also perturbe the prcp_fac or temp_bias the same way as the melt_f

gdir.write_json(params, 'mb_calib', filesuffix='_smaller_melt_f')  # write a new file, with perturbed parameters
tasks.run_from_climate_data(gdir,
                            output_filesuffix='_smaller_melt_f',
                            mb_model_class = partial(massbalance.MonthlyTIModel,
                                                     mb_params_filesuffix='_smaller_melt_f')
                           );

<div class="alert alert-warning">
    <b>Task</b>: Copy the code above and create a model run with a perturbed precipitation factor or the temperature bias!
</div>

In [ ]:
# add your code here

After creating our sensitivity experiments, let’s take a look at how these changes influence the glacier’s dynamic volume evolution:

In [ ]:
# define a function for easy opening of the data
def get_model_diagnostics(gdir, filesuffix):
    with xr.open_dataset(gdir.get_filepath('model_diagnostics', filesuffix=filesuffix)) as ds:
        ds = ds.load()
    return ds

# open the data for our different runs
ds_default = get_model_diagnostics(gdir, filesuffix='_default')
ds_glen_a_doubled = get_model_diagnostics(gdir, filesuffix='_glen_a_doubled')
ds_smaller_melt_f = get_model_diagnostics(gdir, filesuffix='_smaller_melt_f')

# plot the results
ds_default.volume_m3.plot(label='Default');
ds_glen_a_doubled.volume_m3.plot(label='Glen A doubled');
ds_smaller_melt_f.volume_m3.plot(label='Smaller melt factor');
plt.legend()

<div class="alert alert-warning">
    <b>Task</b>: Can you explain why the volume evolution changed the way it did? (Think about how each parameter influences melt, accumulation, and glacier dynamics.)
</div>

Your answer here:

<div class="alert alert-warning">
    <b>Task</b>: Can you find a precipitation factor that brings the volume evolution back in line with the default run, but using a smaller melt factor?
    
Before you start, try to guess: in which direction does the precipitation factor need to change?
</div>

In [ ]:
params = gdir.read_json('mb_calib', filesuffix='_smaller_melt_f')

# search here for a factor to compensate for the smaller melt factor
params['prcp_fac'] *= 1  

gdir.write_json(params, 'mb_calib', filesuffix='_smaller_melt_f_and_adapted_prcp_fac')  # write a new file, with perturbed parameters
tasks.run_from_climate_data(gdir,
                            output_filesuffix='_smaller_melt_f_and_adapted_prcp_fac',
                            mb_model_class = partial(massbalance.MonthlyTIModel,
                                                     mb_params_filesuffix='_smaller_melt_f_and_adapted_prcp_fac')
                           )

ds_smaller_melt_f_and_adapted_prcp_fac = get_model_diagnostics(gdir, filesuffix='_smaller_melt_f_and_adapted_prcp_fac')

# plot the results
ds_default.volume_m3.plot(label='Default');
ds_smaller_melt_f.volume_m3.plot(label='Smaller melt factor');
ds_smaller_melt_f_and_adapted_prcp_fac.volume_m3.plot(label='Smaller melt factor and adapted prcipitation factor');
plt.legend()

<div class="alert alert-warning">
    <b>Task</b>: Why is it possible to find a precipitation factor that compensates for a smaller melt factor, resulting in almost the same volume evolution? (Hint: Think back to the session on mass balance model calibration.)
</div>

Your answer here:

 <div class="alert alert-warning">
    <b>Bonus Task</b>: Can you also compensate for doubling the Glen A parameter by adjusting some of the mass balance parameters?
</div>

In [ ]:
params = gdir.read_json('mb_calib')

params['prcp_fac'] *= 1.2  # here you can play with the precipitation factor
params['melt_f'] *= 1.1  # here you can play with the melt factor

gdir.write_json(params, 'mb_calib', filesuffix='_compensate_larger_glen_a')  # write a new file, with perturbed parameters
tasks.run_from_climate_data(gdir,
                            output_filesuffix='_compensate_larger_glen_a',
                            mb_model_class = partial(massbalance.MonthlyTIModel,
                                                     mb_params_filesuffix='_compensate_larger_glen_a'),
                            glen_a_fac=2
                           )

ds_compensate_larger_glen_a = get_model_diagnostics(gdir, filesuffix='_compensate_larger_glen_a')

# plot the results
ds_default.volume_m3.plot(label='Default');
ds_glen_a_doubled.volume_m3.plot(label='Glen A doubled');
ds_compensate_larger_glen_a.volume_m3.plot(label='Compansate change of glen a');
plt.legend()

## Recape

- We have now recreated all steps of the OGGM default preprocessed directories.
- When accounting for changing glacier geometry, we need to recalibrate the mass balance model accordingly.
- There is an equifinality problem in the mass balance parameters: with only one observation available, multiple parameter combinations can produce similar results.